# City and Weather API's

## City API

In [21]:
import pandas as pd
import os
import plotly.express as px
import requests
from config import AllPath

all_paths = AllPath()

local_project_path = all_paths.project_path
path = local_project_path+'datas'

chemin_fichier_csv = path+"./city.csv"
df = pd.read_csv(chemin_fichier_csv, header=None, names=['city','nan'])
cities = df['city'].str.replace(' ', '-').tolist()

# list(cities)

In [22]:
api_url = 'https://nominatim.openstreetmap.org/search'

enriched_cities = []

for city in cities:

    payload = {
        'city' : city,
        'limit' : 1,
        'country' : 'france',
        'country-code' : 'fr',
        'format' : 'json'
        }
    
    r_city = requests.get(api_url, 
                          params=payload)

    enriched_cities.append({'city': city,
                            'lat' : float(r_city.json()[0]['lat']),
                            'lon' : float(r_city.json()[0]['lon'])})
    
# display(enriched_cities)


In [23]:
df = pd.DataFrame.from_dict(enriched_cities)

In [24]:

fig = px.scatter_mapbox(df,
                        lat='lat',
                        lon='lon',
                        center=dict(lat=46.2, lon=2.2),
                        zoom=4,
                        mapbox_style='carto-positron',
                        hover_name='city',
                        height=600)

fig.show()

## Weather API

In [25]:
print(os.getcwd())


c:\Users\antoi\Documents\Work&Learn\JEDHA\M03-DataCollection_Managment\JEDHA-Projet-2-Kayak


In [26]:

with open("API_key.txt", "r") as file:
    API_key = str(file.readline())

exclude = 'current,minutely,hourly,alerts'
weather_cities = []

for city in enriched_cities:
    lat = city['lat']
    lon = city['lon']
    url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&exclude={exclude}&appid={API_key}&units=metric'
    weather_raw = requests.get(url).json()
    weather_raw['city']['name'] = city['city']
    weather_cities.append(weather_raw)

# display(weather_cities)

In [27]:
pd.json_normalize(weather_cities).head()

,cod,message,cnt,list,city.id,city.name,city.coord.lat,city.coord.lon,city.country,city.population,city.timezone,city.sunrise,city.sunset
0,200,0,40,"[{'dt': 1712221200, 'main': {'temp': 12.94, 'f...",6435453,Mont-Saint-Michel,48.6360,-1.5115,FR,0,7200,1712209012,1712256037
1,200,0,40,"[{'dt': 1712221200, 'main': {'temp': 12.44, 'f...",2978640,St-Malo,48.6495,-2.0260,FR,50676,7200,1712209135,1712256161
2,200,0,40,"[{'dt': 1712221200, 'main': {'temp': 13, 'feel...",3034483,Bayeux,49.2765,-0.7025,FR,15963,7200,1712208777,1712255883
3,200,0,40,"[{'dt': 1712221200, 'main': {'temp': 12.36, 'f...",3003796,Le-Havre,49.4939,0.1080,FR,185972,7200,1712208569,1712255703
4,200,0,40,"[{'dt': 1712221200, 'main': {'temp': 12.69, 'f...",2982652,Rouen,49.4405,1.0940,FR,112787,7200,1712208336,1712255462


In [28]:
pd.json_normalize(weather_cities)['city.name']
# pd.json_normalize(weather_cities)['list']
# pd.json_normalize(pd.json_normalize(weather_cities)['list'][34])
# pd.json_normalize(pd.json_normalize(weather_cities)['list'][34])['weather'][0]

0                Mont-Saint-Michel
1                          St-Malo
2                           Bayeux
3                         Le-Havre
4                            Rouen
5                            Paris
6                           Amiens
7                            Lille
8                       Strasbourg
9     Chateau-du-Haut-Koenigsbourg
10                          Colmar
11                       Eguisheim
12                        Besancon
13                           Dijon
14                          Annecy
15                        Grenoble
16                            Lyon
17                Gorges-du-Verdon
18              Bormes-les-Mimosas
19                          Cassis
20                       Marseille
21                 Aix-en-Provence
22                         Avignon
23                            Uzes
24                           Nimes
25                   Aigues-Mortes
26        Saintes-Maries-de-la-mer
27                       Collioure
28                  

In [29]:
weather_datas = pd.DataFrame()

for id, city in enumerate(pd.json_normalize(weather_cities)['city.name']):

    df3 = pd.json_normalize(pd.json_normalize(weather_cities)['list'][id])
    df3['city'] = city
    df3['lat'] = pd.json_normalize(weather_cities)["city.coord.lat"][id]
    df3['lon'] = pd.json_normalize(weather_cities)["city.coord.lon"][id]
    
    weather_datas = pd.concat([weather_datas, df3])

weather_datas['dt_txt'] = pd.to_datetime(weather_datas['dt_txt']) 
display(weather_datas.head(3))

,dt,weather,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,...,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,rain.3h,sys.pod,city,lat,lon
0,1712221200,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000,0.34,2024-04-04 09:00:00,12.94,12.59,12.94,12.95,1007,...,-0.01,83,7.61,233,12.85,0.20,d,Mont-Saint-Michel,48.636,-1.5115
1,1712232000,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000,0.20,2024-04-04 12:00:00,14.72,14.31,14.72,15.61,1009,...,-0.89,81,8.41,241,13.06,0.20,d,Mont-Saint-Michel,48.636,-1.5115
2,1712242800,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000,1.00,2024-04-04 15:00:00,13.27,13.03,13.27,13.27,1011,...,0.00,100,5.47,224,10.83,0.62,d,Mont-Saint-Michel,48.636,-1.5115


In [30]:
col_drop = ['dt', 'weather', 'visibility', 'main.temp', 'main.temp_min', 'main.temp_max', 'main.pressure','main.sea_level',
            'main.grnd_level', 'main.humidity', 'main.temp_kf', 'clouds.all', 'wind.deg', 'wind.gust', 'rain.3h']

weather_datas.drop(columns=col_drop, inplace=True)


In [31]:
weather_during_day = weather_datas[weather_datas['sys.pod']=='d'].drop(columns='sys.pod')

In [32]:
weather_day_mean = weather_during_day.groupby('city').mean().reset_index()

In [33]:
display(weather_day_mean[['city','pop']].sort_values('pop').head())
display(weather_day_mean[['city','main.feels_like']].sort_values('main.feels_like',ascending=False).head())
display(weather_day_mean[['city','wind.speed']].sort_values('wind.speed').head())

,city,pop
24,Marseille,0.0000
10,Bormes-les-Mimosas,0.0000
14,Collioure,0.0000
30,Saintes-Maries-de-la-mer,0.0084
12,Cassis,0.0124


,city,main.feels_like
4,Ariege,20.1508
19,Grenoble,19.8328
1,Aix-en-Provence,18.3572
5,Avignon,18.0036
23,Lyon,17.9784


,city,wind.speed
3,Annecy,1.8172
18,Gorges-du-Verdon,2.1900
13,Chateau-du-Haut-Koenigsbourg,2.3576
14,Collioure,2.6972
4,Ariege,2.7008


Create a SCORE to select the city

In [34]:
weights = {'pop': -0.4, 'main.feels_like': 0.5, 'wind.speed': -0.1}

weather_day_mean['score'] = (weather_day_mean['pop'] * weights['pop'] +
                             weather_day_mean['main.feels_like'] * weights['main.feels_like'] +
                             weather_day_mean['wind.speed'] * weights['wind.speed'])

min_score = weather_day_mean['score'].min()
max_score = weather_day_mean['score'].max()
weather_day_mean['score'] = 10 * (weather_day_mean['score'] - min_score) / (max_score - min_score)

display(weather_day_mean.sort_values('score',ascending=False).head())

,city,pop,dt_txt,main.feels_like,wind.speed,lat,lon,score
4,Ariege,0.0408,2024-04-06 16:48:00,20.1508,2.7008,42.8458,1.6068,10.000000
19,Grenoble,0.0400,2024-04-06 16:48:00,19.8328,2.8200,45.1876,5.7358,9.608137
1,Aix-en-Provence,0.0144,2024-04-06 16:48:00,18.3572,3.2436,43.5298,5.4475,7.839653
15,Colmar,0.1304,2024-04-06 16:48:00,17.9720,2.8680,48.0778,7.3580,7.376951
5,Avignon,0.0400,2024-04-06 16:48:00,18.0036,3.6424,43.9492,4.8059,7.318424


In [35]:
fig = px.scatter_mapbox(weather_day_mean,
                        lat='lat',
                        lon='lon',
                        color='score',
                        size='score',
                        center=dict(lat=47, lon=2.2),
                        zoom=4.5,
                        color_continuous_scale=['white', 'yellow','orange', 'red','purple'],
                        mapbox_style='carto-positron',
                        hover_name='city',
                        hover_data=['main.feels_like', "pop", "wind.speed", "score"],
                        height=600,
                        width=600*16/9)

fig.show()

In [36]:
top5_cities = weather_day_mean.sort_values('score',ascending=False).head()
top5_cities

,city,pop,dt_txt,main.feels_like,wind.speed,lat,lon,score
4,Ariege,0.0408,2024-04-06 16:48:00,20.1508,2.7008,42.8458,1.6068,10.000000
19,Grenoble,0.0400,2024-04-06 16:48:00,19.8328,2.8200,45.1876,5.7358,9.608137
1,Aix-en-Provence,0.0144,2024-04-06 16:48:00,18.3572,3.2436,43.5298,5.4475,7.839653
15,Colmar,0.1304,2024-04-06 16:48:00,17.9720,2.8680,48.0778,7.3580,7.376951
5,Avignon,0.0400,2024-04-06 16:48:00,18.0036,3.6424,43.9492,4.8059,7.318424


In [37]:

fig = px.scatter_mapbox(top5_cities,
                        lat='lat',
                        lon='lon',
                        color='score',
                        size='score',
                        center=dict(lat=top5_cities['lat'].mean(), lon=top5_cities['lon'].mean()),
                        zoom=6,
                        range_color=(0, 10),
                        color_continuous_scale=['white', 'yellow','orange', 'red','purple'],
                        mapbox_style='carto-positron',
                        text='city',
                        hover_name='city',
                        hover_data=['main.feels_like', "pop", "wind.speed", "score"],
                        height=600,
                        width=600*16/9)


fig.show()

In [38]:
weather_day_mean.rename(columns={
    'city': 'city_target',
    'pop': 'city_target_precipitation',
    'dt_txt': 'date_time',
    'main.feels_like': 'city_target_temp_feels_like',
    'wind.speed': 'city_target_wind_speed',
    'lat': 'lat_target',
    'lon': 'lon_target',
    'score': 'city_target_weather_score',
}, inplace=True)

In [39]:
weather_day_mean.sort_values('city_target_weather_score',ascending=False)

,city_target,city_target_precipitation,date_time,city_target_temp_feels_like,city_target_wind_speed,lat_target,lon_target,city_target_weather_score
4,Ariege,0.0408,2024-04-06 16:48:00,20.1508,2.7008,42.8458,1.6068,10.000000
19,Grenoble,0.0400,2024-04-06 16:48:00,19.8328,2.8200,45.1876,5.7358,9.608137
1,Aix-en-Provence,0.0144,2024-04-06 16:48:00,18.3572,3.2436,43.5298,5.4475,7.839653
15,Colmar,0.1304,2024-04-06 16:48:00,17.9720,2.8680,48.0778,7.3580,7.376951
5,Avignon,0.0400,2024-04-06 16:48:00,18.0036,3.6424,43.9492,4.8059,7.318424
17,Eguisheim,0.1404,2024-04-06 16:48:00,17.6920,2.8316,48.0448,7.3080,7.054548
3,Annecy,0.0348,2024-04-06 16:48:00,17.3312,1.8172,45.8992,6.1289,6.970204
14,Collioure,0.0000,2024-04-06 16:48:00,17.4180,2.6972,42.5251,3.0832,6.899733
23,Lyon,0.0820,2024-04-06 16:48:00,17.9784,5.7356,45.7578,4.8320,6.770092
7,Bayonne,0.0460,2024-04-06 16:48:00,17.2660,3.3132,43.4945,-1.4737,6.541405


In [41]:
weather_day_mean.to_csv(all_paths.save_data_path+'cities_weather_datas.csv',index=False)